## Capstone final project week 5

In [3]:
#Importing packages 
import pandas as pd
import numpy as np
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

wikipedia="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki=requests.get(wikipedia)
wiki_lst=pd.read_html(wiki.content,header=0)[0]
df = wiki_lst[wiki_lst.Neighbourhood != 'Not assigned']
df.reset_index(inplace = True)
df.head(12)


,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,9,M1B,Scarborough,"Malvern, Rouge"
7,11,M3B,North York,Don Mills
8,12,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
toronto_df_dropna = df[df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_grouped = toronto_df_dropna.groupby(["Postal Code", "Borough"], as_index=False).first()
toronto_df_grouped.drop(['index'],axis=1,inplace=True)

In [5]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
        
toronto_df_grouped.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
toronto_df_grouped.shape


(103, 3)

In [8]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
toronto_df_new = toronto_df_grouped.merge(coordinates, on="Postal Code")
toronto_df_new.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [10]:
! pip install geopy

In [11]:
from geopy.geocoders import Nominatim 
address = 'Toronto'

geolocator = Nominatim(user_agent="my_app")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [12]:
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: ...working... done

# All requested packages already installed.



In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [13]:
# filter borough names that contain the word Toronto
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [14]:
# create a new DataFrame with only boroughs that contain the word Toronto
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [40]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [16]:
CLIENT_ID = 'AKGIXI5ZWFJEIY5VWFX2KYMXBCHAAYFILFZTA5DTP5K3ZH2N' 
CLIENT_SECRET = 'SMTXUWTRF4SO10EGWCQZHOHW1CXYMLFPATOXZ3P5JLES0THW' 
VERSION = '20180605' 

radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighbourhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Postal Code'], toronto_df_new['Borough'], toronto_df_new['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighbourhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [17]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1630, 9)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [18]:
venues_df["VenueCategory"].value_counts()

Coffee Shop           159
Café                   90
Restaurant             54
Italian Restaurant     38
Hotel                  37
                     ... 
Harbor / Marina         1
Bed & Breakfast         1
Chocolate Shop          1
Swim School             1
College Rec Center      1
Name: VenueCategory, Length: 234, dtype: int64

In [19]:
#venues_df=venues_df[venues_df.VenueCategory=='Pub' and ]
#venues_df

In [20]:
#Let's check how many venues were returned for each PostalCode

venues_df.groupby(["Postal Code", "Borough", "Neighbourhood"]).count()

,,,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Postal Code,Borough,Neighbourhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale",42,42,42,42,42,42
M4L,East Toronto,"India Bazaar, The Beaches West",18,18,18,18,18,18
M4M,East Toronto,Studio District,40,40,40,40,40,40
M4N,Central Toronto,Lawrence Park,3,3,3,3,3,3
M4P,Central Toronto,Davisville North,7,7,7,7,7,7
M4R,Central Toronto,"North Toronto West, Lawrence Park",18,18,18,18,18,18
M4S,Central Toronto,Davisville,35,35,35,35,35,35
M4T,Central Toronto,"Moore Park, Summerhill East",3,3,3,3,3,3


In [21]:
venues_df['VenueCategory'].unique()[:50]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Cosmetics Shop', 'Ice Cream Shop', 'Greek Restaurant',
       'Italian Restaurant', 'Brewery', 'Juice Bar', 'Yoga Studio',
       'Fruit & Vegetable Store', 'Dessert Shop', 'Pizza Place',
       'Restaurant', 'Bookstore', 'Café', 'Furniture / Home Store', 'Spa',
       'Grocery Store', 'Bubble Tea Shop', 'Coffee Shop', 'Bakery',
       'Caribbean Restaurant', 'Indian Restaurant', 'Lounge',
       'Frozen Yogurt Shop', 'Liquor Store', 'American Restaurant',
       'Sushi Restaurant', 'Gym', 'Fish & Chips Shop',
       'Fast Food Restaurant', 'Park', 'Burrito Place', 'Pet Store',
       'Steakhouse', 'Movie Theater', 'Sandwich Place', 'Fish Market',
       'Seafood Restaurant', 'Gay Bar', 'Cheese Shop',
       'Middle Eastern Restaurant', 'Comfort Food Restaurant',
       'Stationery Store', 'Thai Restaurant', 'Coworking Space',
       'Wine Bar', 'Latin American Restaurant'], dtype=object)

### Analyze Each Area

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['Postal Code'] = venues_df['Postal Code'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighbourhoods'] = venues_df['Neighbourhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1630, 237)


,Postal Code,Borough,Neighbourhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West, Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
to_grouped = toronto_onehot.groupby(["Neighbourhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped



(39, 235)


,Neighbourhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.016949,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.016129,0.000000,0.000000,0.016129,0.000000,0.00,0.016129
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Church and Wellesley,0.013158,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.013158,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.026316
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


In [24]:
pub = toronto_onehot[["Neighbourhoods","Pub"]]
pub

,Neighbourhoods,Pub
0,The Beaches,0
1,The Beaches,0
2,The Beaches,1
3,The Beaches,0
4,"The Danforth West, Riverdale",0
...,...,...
1625,"Business reply mail Processing Centre, South C...",0
1626,"Business reply mail Processing Centre, South C...",0
1627,"Business reply mail Processing Centre, South C...",0
1628,"Business reply mail Processing Centre, South C...",0


In [25]:
# set number of clusters
toclusters = 2

to_clustering = pub.drop(["Neighbourhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [26]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = pub.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_
to_merged

,Neighbourhoods,Pub,Cluster Labels
0,The Beaches,0,0
1,The Beaches,0,0
2,The Beaches,1,1
3,The Beaches,0,0
4,"The Danforth West, Riverdale",0,0
...,...,...,...
1625,"Business reply mail Processing Centre, South C...",0,0
1626,"Business reply mail Processing Centre, South C...",0,0
1627,"Business reply mail Processing Centre, South C...",0,0
1628,"Business reply mail Processing Centre, South C...",0,0


In [27]:
to_merged.rename(columns={"Neighbourhoods": "Neighbourhood"}, inplace=True)
to_merged = to_merged.join(venues_df.set_index("Neighbourhood"), on="Neighbourhood")
print(to_merged.shape)
to_merged.head()

(112434, 11)


,Neighbourhood,Pub,Cluster Labels,Postal Code,Borough,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,The Beaches,0,0,M4E,East Toronto,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
0,The Beaches,0,0,M4E,East Toronto,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
0,The Beaches,0,0,M4E,East Toronto,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
0,The Beaches,0,0,M4E,East Toronto,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
1,The Beaches,0,0,M4E,East Toronto,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail


In [28]:
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

(112434, 11)


,Neighbourhood,Pub,Cluster Labels,Postal Code,Borough,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,The Beaches,0,0,M4E,East Toronto,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1008,"Commerce Court, Victoria Hotel",0,0,M5L,Downtown Toronto,43.648198,-79.379817,Craft Beer Market,43.649872,-79.378398,Beer Bar
1008,"Commerce Court, Victoria Hotel",0,0,M5L,Downtown Toronto,43.648198,-79.379817,Movenpick Cafe,43.647687,-79.377295,Café
1008,"Commerce Court, Victoria Hotel",0,0,M5L,Downtown Toronto,43.648198,-79.379817,King Taps,43.648476,-79.382058,Gastropub
1008,"Commerce Court, Victoria Hotel",0,0,M5L,Downtown Toronto,43.648198,-79.379817,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
970,"Commerce Court, Victoria Hotel",1,1,M5L,Downtown Toronto,43.648198,-79.379817,Crepe TO,43.650063,-79.374587,Creperie
970,"Commerce Court, Victoria Hotel",1,1,M5L,Downtown Toronto,43.648198,-79.379817,DAVIDsTEA,43.650547,-79.383385,Tea Room
970,"Commerce Court, Victoria Hotel",1,1,M5L,Downtown Toronto,43.648198,-79.379817,The Poké Box,43.650469,-79.376317,Poke Place
970,"Commerce Court, Victoria Hotel",1,1,M5L,Downtown Toronto,43.648198,-79.379817,Hudson's Bay,43.652040,-79.380391,Department Store


In [29]:
to_merged.loc[to_merged['Cluster Labels'] == 0]

,Neighbourhood,Pub,Cluster Labels,Postal Code,Borough,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,The Beaches,0,0,M4E,East Toronto,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1008,"Commerce Court, Victoria Hotel",0,0,M5L,Downtown Toronto,43.648198,-79.379817,Craft Beer Market,43.649872,-79.378398,Beer Bar
1008,"Commerce Court, Victoria Hotel",0,0,M5L,Downtown Toronto,43.648198,-79.379817,Movenpick Cafe,43.647687,-79.377295,Café
1008,"Commerce Court, Victoria Hotel",0,0,M5L,Downtown Toronto,43.648198,-79.379817,King Taps,43.648476,-79.382058,Gastropub
1008,"Commerce Court, Victoria Hotel",0,0,M5L,Downtown Toronto,43.648198,-79.379817,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
615,Central Bay Street,0,0,M5G,Downtown Toronto,43.657952,-79.387383,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
615,Central Bay Street,0,0,M5G,Downtown Toronto,43.657952,-79.387383,Chatime 日出茶太,43.655542,-79.384684,Bubble Tea Shop
615,Central Bay Street,0,0,M5G,Downtown Toronto,43.657952,-79.387383,Hailed Coffee,43.658833,-79.383684,Coffee Shop
615,Central Bay Street,0,0,M5G,Downtown Toronto,43.657952,-79.387383,Somethin' 2 Talk About,43.658395,-79.385338,Middle Eastern Restaurant


In [30]:
to_merged.loc[to_merged['Cluster Labels'] == 1]


,Neighbourhood,Pub,Cluster Labels,Postal Code,Borough,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
341,"Regent Park, Harbourfront",1,1,M5A,Downtown Toronto,43.654260,-79.360636,Wine Rack,43.656573,-79.356928,Wine Shop
177,"Summerhill West, Rathnelly, South Hill, Forest...",1,1,M4V,Central Toronto,43.686412,-79.400049,The Market By Longo’s,43.686711,-79.399536,Supermarket
177,"Summerhill West, Rathnelly, South Hill, Forest...",1,1,M4V,Central Toronto,43.686412,-79.400049,LCBO,43.686991,-79.399238,Liquor Store
341,"Regent Park, Harbourfront",1,1,M5A,Downtown Toronto,43.654260,-79.360636,The Beer Store,43.657773,-79.357463,Beer Store
177,"Summerhill West, Rathnelly, South Hill, Forest...",1,1,M4V,Central Toronto,43.686412,-79.400049,Mary Be Kitchen,43.687708,-79.395062,Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
970,"Commerce Court, Victoria Hotel",1,1,M5L,Downtown Toronto,43.648198,-79.379817,Crepe TO,43.650063,-79.374587,Creperie
970,"Commerce Court, Victoria Hotel",1,1,M5L,Downtown Toronto,43.648198,-79.379817,DAVIDsTEA,43.650547,-79.383385,Tea Room
970,"Commerce Court, Victoria Hotel",1,1,M5L,Downtown Toronto,43.648198,-79.379817,The Poké Box,43.650469,-79.376317,Poke Place
970,"Commerce Court, Victoria Hotel",1,1,M5L,Downtown Toronto,43.648198,-79.379817,Hudson's Bay,43.652040,-79.380391,Department Store


In [35]:
#cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 0, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]


,Pub,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,0,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1008,0,43.648198,-79.379817,Craft Beer Market,43.649872,-79.378398,Beer Bar
1008,0,43.648198,-79.379817,Movenpick Cafe,43.647687,-79.377295,Café
1008,0,43.648198,-79.379817,King Taps,43.648476,-79.382058,Gastropub
1008,0,43.648198,-79.379817,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant
...,...,...,...,...,...,...,...
615,0,43.657952,-79.387383,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
615,0,43.657952,-79.387383,Chatime 日出茶太,43.655542,-79.384684,Bubble Tea Shop
615,0,43.657952,-79.387383,Hailed Coffee,43.658833,-79.383684,Coffee Shop
615,0,43.657952,-79.387383,Somethin' 2 Talk About,43.658395,-79.385338,Middle Eastern Restaurant


In [37]:
#cluster 2
to_merged.loc[to_merged['Cluster Labels'] == 1, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]


,Pub,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
341,1,43.654260,-79.360636,Wine Rack,43.656573,-79.356928,Wine Shop
177,1,43.686412,-79.400049,The Market By Longo’s,43.686711,-79.399536,Supermarket
177,1,43.686412,-79.400049,LCBO,43.686991,-79.399238,Liquor Store
341,1,43.654260,-79.360636,The Beer Store,43.657773,-79.357463,Beer Store
177,1,43.686412,-79.400049,Mary Be Kitchen,43.687708,-79.395062,Restaurant
...,...,...,...,...,...,...,...
970,1,43.648198,-79.379817,Crepe TO,43.650063,-79.374587,Creperie
970,1,43.648198,-79.379817,DAVIDsTEA,43.650547,-79.383385,Tea Room
970,1,43.648198,-79.379817,The Poké Box,43.650469,-79.376317,Poke Place
970,1,43.648198,-79.379817,Hudson's Bay,43.652040,-79.380391,Department Store


## Observation

### In cluster 2, pubs are more so its easy to say that the chances of drug supply are more and neighbourhoods can be located according to the intensity of drug usage. Whereas in cluster 1, pubs are less and public parks and markets are more. So, less probability that DAE could find the drug dealers 